In [1]:
import pandas as pd
from dateutil import parser
import datetime

In [2]:
cols = ["Country", "Program", "Parent ICA", "Reported/Processed", "OrigMeasure", "Period", "Value"]
df = pd.read_csv("c:\\Users\\tbrowne\\Python\\CleanDriverFile.csv", usecols=cols)

# Convert their date format to a standard date
df["Month"] = df["Period"].apply(parser.parse)

# Keep only the relevant columns
cols = ["Country", "Program", "Parent ICA", "Reported/Processed", "OrigMeasure", "Month", "Value"]
df = df[cols]

###### Select only the top 12 countries

In [3]:
before = df.shape[0]
countries = ['United Arab Emirates', 'South Africa', 'Saudi Arabia', 'Nigeria', 'Egypt',\
            'Qatar', 'Lebanon', 'Kuwait', 'Pakistan', 'Jordan','Libya', 'Tunisia']
df = df.loc[df["Country"].isin(countries)]
print("{:,.0f} rows removed due to Country filter".format(before - df.shape[0]))

677,618 rows removed due to Country filter


###### Split into Reported and Processed

In [4]:
rept = df.loc[df["Reported/Processed"]=="Reported"]
proc = df.loc[df["Reported/Processed"]=="Processed"]
assert (rept.shape[0] + proc.shape[0] == df.shape[0])
del rept["Reported/Processed"]
del proc["Reported/Processed"]

###### "Reported"

In [5]:
output = open("Loss.csv", "w")

In [6]:
def reportLoss(idx, typ):
    rec = ",".join(idx)
    rec = rec + ","+ typ +","+ "\n"
    output.write(rec)

In [7]:
def removeLoss(df, cutoff, typ):
    sort = ["Country", "Program", "Parent ICA", "OrigMeasure", "Month"]
    df = df.sort_values(sort)
    df = df.reset_index(drop=True)
    df = df.set_index(["Country", "Program", "Parent ICA", "OrigMeasure"], drop=True)

    cutoff = datetime.datetime.strptime(cutoff, "%Y-%m-%d")

    for idx, val in df.groupby(level=[0, 1, 2, 3]):
        dates = val["Month"]
        if dates.iloc[-1] < cutoff:
            reportLoss(idx, typ)
            df = df.drop(idx)

    return df.reset_index()

In [8]:
rept = removeLoss(rept, "2017-08-01", "P")

In [9]:
proc = removeLoss(proc, "2017-06-30", "R")

output.close()

In [10]:
rept.to_csv("Reported.csv", sep=",", index=False)
proc.to_csv("Processed.csv", sep=",", index=False)

PermissionError: [Errno 13] Permission denied: 'Processed.csv'